# End-to-End Evaluations

We construct an end-to-end RAG pipeline: 
* Parse unstructured data from PDFs leveraging the raw-text extraction approach
* Employ OpenAI’s text-embedding-3-large model to index and retrieve relevant data chunks. 
* In the generation phase, we incorporate the Chain-of-Thought approach to handle arithmetic-intensive tasks.

 Based on this end-to-end pipeline, we evaluate 8 LLMs spanning various model sizes and architectures.

In [1]:
# Preparation
import sys
import os
from pathlib import Path

# Get the project root directory
root_dir = Path(os.path.abspath("")).resolve().parents[1]
sys.path.append(str(root_dir))
# Change the working directory to the project root
os.chdir(root_dir)

res_dir = f"experiment/e2e/res/"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
    
import warnings
warnings.filterwarnings('ignore')

In our paper, we use the powerful `text-embedding-3-large-model` model with AzureOpenAI-API . But you need to set up with your own api-key, endpoint and deploy-model in the config_file [uda/utils/access_config.py](../../uda/utils/access_config.py). 

If you want to use the API from **other alternative platforms** please change the codes in [uda/utils/retrieve.py (line-80)](../../uda/utils/retrieve.py#L81). 

For convenient demonstration, we choose the `colbert` retriever here.

In [2]:
# Experimental Configurations

# Available retrieval model_name: "bm25", "all-MiniLM-L6-v2", "all-mpnet-base-v2", "openai", "colbert"
# We choose bm25 for convenience
RT_MODEL = "colbert" 

DATASET_NAME_LIST = ["fin", "feta", "tat","paper_text", "nq", "paper_tab"]
LOCAL_LLM_DICT = {
    "meno-tiny": "bond005/meno-tiny-0.1",
    "qwen2.5-1.5B": "Qwen/Qwen2.5-1.5B-Instruct",
    "qwen2.5-3B": "Qwen/Qwen2.5-3B-Instruct",
    "falcon-e-3B": "tiiuae/Falcon-E-3B-Instruct"
}
LLM_LIST = ["meno-tiny", "qwen2.5-1.5B", "qwen2.5-3B", "falcon-e-3B"]

In our implementation, the **AzureOpenAI-API** serves as the interface for accessing GPT models. Users should set up the gpt-service with their own api-key and endpoint in the config_file [uda/utils/access_config.py](../../uda/utils/access_config.py). These configurations will be used in the `call_gpt()` function in the following codes.


If you want to use **other alternative platforms**, the `call_gpt()` can be replaced by the corresponding model-calling function.

In [3]:
from uda.utils import retrieve as rt
from uda.utils import preprocess as pre
import pandas as pd
from uda.utils import llm
from uda.utils import inference
import json

for DATASET_NAME in DATASET_NAME_LIST:
    for LLM_MODEL in LLM_LIST:
        print(f"=== Start {DATASET_NAME} on {LLM_MODEL} ===")
        res_file = os.path.join(res_dir, f"{DATASET_NAME}_{LLM_MODEL}_{RT_MODEL}.jsonl")

        # If use the local LLM, initialize the model
        if LLM_MODEL in LOCAL_LLM_DICT:
            llm_name = LOCAL_LLM_DICT[LLM_MODEL]
            llm_service = inference.LLM(llm_name)
            llm_service.init_llm()

        # Load the benchmark data
        bench_json_file = pre.meta_data[DATASET_NAME]["bench_json_file"]
        with open(bench_json_file, "r") as f:
            bench_data = json.load(f)

        # Run experiments on the demo docs
        doc_list = list(bench_data.keys())
        for doc in doc_list:
            pdf_path = pre.get_example_pdf_path(DATASET_NAME, doc)
            if pdf_path is None:
                continue
            # Prepare the index for the document
            collection_name = f"{DATASET_NAME}_vector_db"
            collection = rt.prepare_collection(pdf_path, collection_name, RT_MODEL)
            for qa_item in bench_data[doc]:
                question = qa_item["question"]
                # Retrieve the contexts
                contexts = rt.get_contexts(collection, question, RT_MODEL)
                context_text = '\n'.join(contexts)
                # Create the prompt
                llm_message = llm.make_prompt(question, context_text, DATASET_NAME, LLM_MODEL)
                # Generate the answer
                if LLM_MODEL in LOCAL_LLM_DICT:
                    response = llm_service.infer(llm_message)
                elif LLM_MODEL == "gpt4":
                    # Set up with your own GPT4 service using environment variables
                    response = llm.call_gpt(messages=llm_message)
                    if response is None:
                        print("Make sure your gpt4 service is set up correctly.")
                        raise Exception("GPT4 service")

                # log the results
                res_dict = {"model": LLM_MODEL, "question": question, "response": response, "doc": doc, "q_uid": qa_item["q_uid"], "answers": qa_item["answers"]}
                print(res_dict)
                with open(res_file, "a") as f:
                    f.write(json.dumps(res_dict) + "\n")
            rt.reset_collection(collection_name, RT_MODEL)

    print(f"=== Finish {DATASET_NAME} ===\n")


=== Start fin on meno-tiny ===
2025-07-29 14:43:09 ====== Init LLM =======


2025-07-29 14:43:10,143 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Device set to use cuda:0


2025-07-29 14:43:13 ====== LLM Service Started =======
---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Jul 29, 14:43:15] #> Note: Output directory .ragatouille/colbert/indexes/fin_vector_db already exists


[Jul 29, 14:43:15] #> Will delete 10 files already at .ragatouille/colbert/indexes/fin_vector_db in 20 seconds...
[Jul 29, 14:43:38] [0] 		 #> Encoding 50 passages..
[Jul 29, 14:43:38] [0] 		 avg_doclen_est = 3.819999933242798 	 len(local_sample) = 50
[Jul 29, 14:43:38] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/fin_vector_db/plan.json ..
used 4 iterations (0.03

0it [00:00, ?it/s]

[Jul 29, 14:43:39] [0] 		 #> Encoding 50 passages..


...

Loading searcher for index paper_tab_vector_db for the first time... This may take a few seconds
[Jul 29, 15:32:07] #> Loading codec...
[Jul 29, 15:32:07] #> Loading IVF...
[Jul 29, 15:32:07] #> Loading doclens...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4877.10it/s]

[Jul 29, 15:32:07] #> Loading codes and residuals...



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1577.40it/s]
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: What accuracy does the proposed system achieve?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054, 10640,  2515,  1996,  3818,  2291,  6162,  1029,
          102,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

2025-07-29 15:32:07 CallLLM
{'model': 'falcon-e-3B', 'question': 'What accuracy does the proposed system achieve?', 'response': 'The answer is: 90% accuracy', 'doc': '1801.05147', 'q_uid': 'ef4dba073d24042f24886580ae77add5326f2130', 'answers': [{'answer': 'F1 scores of 85.99 on the DL-PS data,  75.15 on the EC-MT data and 71.53 on the EC-UQ data ', 'type': '

### Evaluate the end-to-end results

In [4]:
dataset_name="feta"
llm_model="meno-tiny"
rt_model="colbert"
res_file_name=f"experiment/e2e/res/{dataset_name}_{llm_model}_{rt_model}.jsonl"

from uda.eval.my_eval import eval_from_file
eval_from_file(dataset_name, res_file_name)

{'Answer F1': 0.19444444444444442, 'Missing predictions': 0}


In [9]:
for llm_model in LOCAL_LLM_DICT:
    print('========')
    print(llm_model)
    print('========')
    for dataset_name in DATASET_NAME_LIST:
        res_file_name=f"experiment/e2e/res/{dataset_name}_{llm_model}_{rt_model}.jsonl"
        print(dataset_name + ':')
        eval_from_file(dataset_name, res_file_name)
    print('')

meno-tiny
fin:
Exact-match accuracy: 0.00
feta:
{'Answer F1': 0.19444444444444442, 'Missing predictions': 0}
tat:
Numerical F1 score: 2.83
paper_text:
{'Answer F1': 0.452991452991453, 'Missing predictions': 0}
nq:
{'Answer F1': 0.11717384458198757, 'Missing predictions': 0}
paper_tab:
{'Answer F1': 0.04554079696394687, 'Missing predictions': 0}

qwen2.5-1.5B
fin:
Exact-match accuracy: 0.00
feta:
{'Answer F1': 0.2562162162162162, 'Missing predictions': 0}
tat:
Numerical F1 score: 2.67
paper_text:
{'Answer F1': 0.4892857142857143, 'Missing predictions': 0}
nq:
{'Answer F1': 0.1848040472576633, 'Missing predictions': 0}
paper_tab:
{'Answer F1': 0.022727272727272728, 'Missing predictions': 0}

qwen2.5-3B
fin:
Exact-match accuracy: 0.00
feta:
{'Answer F1': 0.2843980343980344, 'Missing predictions': 0}
tat:
Numerical F1 score: 0.00
paper_text:
{'Answer F1': 0.37931034482758624, 'Missing predictions': 0}
nq:
{'Answer F1': 0.21613524555259797, 'Missing predictions': 0}
paper_tab:
{'Answer F1':